In [22]:
import uproot
import awkward as ak
import matplotlib.pyplot as plt
import hist
from hist import Hist
from coffea.nanoevents import NanoEventsFactory, BaseSchema
import coffea.processor as processor
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
import numpy as np
import mplhep as hep

In [23]:
fileset = "/eos/home-b/bchitrod/HHH/NANOAOD/TRSM_XToHY_6b_M3_2000_M2_1100_NANOAOD.root" 

In [24]:
eventsToRead = 10000
events = NanoEventsFactory.from_root(fileset,schemaclass=NanoAODSchema,metadata={"dataset": "testSignal"},entry_stop=eventsToRead).events()

In [25]:
#useful functions
def closest(masses):
    delta = abs(125 - masses)
    closest_masses = ak.min(delta, axis=1)
    is_closest = (delta == closest_masses)
    return is_closest

def HbbvsQCD(fatjet):
    score = (fatjet.particleNetMD_Xbb/(fatjet.particleNetMD_Xbb+fatjet.particleNetMD_QCD))
    return score


In [26]:


    fatjets = events.FatJet
    jets = events.Jet

    ##FatJet cuts                                                                                                                                                                                                                                     
    ptcut = 250
    etacut = 2.5
    mass_cut = [100,150]
    pNet_cut = 0.9105

    good_fatjets = fatjets[(fatjets.pt>ptcut) & (np.absolute(fatjets.eta)<etacut) & (fatjets.msoftdrop>=mass_cut[0]) & (fatjets.msoftdrop<=mass_cut[1])]
    pre_semiboosted = good_fatjets[ak.num(good_fatjets, axis=1) == 2]
    print(ak.num(pre_semiboosted,axis=0))
    pre_boosted = good_fatjets[ak.num(good_fatjets, axis=1)> 2]
    btag_semiboosted = pre_semiboosted[HbbvsQCD(pre_semiboosted)>=pNet_cut]
    btag_boosted = pre_boosted[HbbvsQCD(pre_boosted)>=pNet_cut]
    good_semiboosted = btag_semiboosted[ak.num(btag_semiboosted, axis=1) == 2]
    good_boosted = btag_boosted[ak.num(btag_boosted, axis=1)> 2]
    print(ak.num(good_semiboosted,axis=0))
    
    res_ptcut = 30
    res_etacut = 2.5
    res_mass_cut = [90,150]
    # loose cut = 0.0532, med_cut = 0.3040, tight_cut = 0.7476 , https://twiki.cern.ch/twiki/bin/view/CMS/BtagRecommendation106XUL17                                                                                                                  
    res_deepBcut = 0.0532

    good_jets = jets[(jets.pt > res_ptcut) & (np.absolute(jets.eta) < res_etacut) & (jets.btagDeepB>res_deepBcut)]

    res_jets = (good_jets[ak.num(btag_semiboosted, axis=1) == 2])
    print(ak.num(res_jets,axis=0))
    good_pairs = res_jets.nearest(good_semiboosted).delta_r(res_jets)>0.8
    good_paired = res_jets[good_pairs]
    res_bjets = (good_paired[ak.num(good_paired, axis=1) >= 2])
    print(ak.num(res_bjets,axis=0))
    dijets = ak.combinations(res_bjets, 2, fields=['i0', 'i1'])
    dijet_masses = (dijets['i0'] + dijets['i1']).mass
    # Get the masses closest to specified value (see function above)                                                                                                                                                                                  
    is_closest = closest(dijet_masses)
    closest_dijets = dijets[is_closest]

    mass_jets = closest_dijets[((closest_dijets['i0'] + closest_dijets['i1']).mass>=res_mass_cut[0]) & ((closest_dijets['i0'] + closest_dijets['i1']).mass<=res_mass_cut[1])]
    selected_jets = mass_jets[ak.num(mass_jets,axis=1)>0]
    print(ak.num(selected_jets,axis=0))
    selected_fatjets = good_semiboosted[ak.num(selected_jets, axis=1)>0]
    semiboosted = ak.num(selected_fatjets,axis=0)

3398
2535
2535
2478
1864


In [27]:
print(semiboosted)

1864


In [28]:
print("Efficiency = ",semiboosted/100)

Efficiency =  18.64
